ANN From Scratch

In [152]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [153]:
df = pd.read_csv("data.csv")

In [158]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [154]:
df.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [155]:
df.drop(['Unnamed: 32', 'id'], axis=1, inplace=True)

In [156]:
x = df.drop('diagnosis', axis=1)
y = df['diagnosis']

In [157]:
lb = LabelEncoder()
y = lb.fit_transform(y)  # Converts 'B', 'M' into 0 and 1

In [142]:
# Split into train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=40)

In [143]:
# Standardize the features
sc = StandardScaler()
xtrain = sc.fit_transform(xtrain)
xtest = sc.transform(xtest)

ANN

- input layer
- hidden layer
- output layer


Key Steps:
1. random weights initialization
2. feed forward
3. back propogation
4. training

In [144]:
# Define activation and loss functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [145]:
def sigmoid_derivative(x):
    return x * (1 - x)

In [146]:
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [147]:
# ANN 
class ANN:
    def __init__(self, input_size, hidden_size, output_size):
        
        # Random weights initialization
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.weights_hidden_output = np.random.rand(hidden_size, output_size)

        #random bias initialization
        self.bias_hidden = np.random.rand(1, hidden_size)
        self.bias_output = np.random.rand(1, output_size)

    # Forward pass
    def forward(self, X):
        self.hidden_layer_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = sigmoid(self.hidden_layer_input)
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.output = sigmoid(self.output_layer_input)
        return self.output

    # Backpropagation
    def backward(self, X, y, learning_rate):
        # Ensuring y is reshaped properly for the output layer
        y = y.reshape(-1, 1)

        # Output layer error and gradient
        
        # Eoutput= y-y^
        output_error = y - self.output

        #outputGradient= Eoutput * sigmoidDerivative(y^)
        output_delta = output_error * sigmoid_derivative(self.output)

        # Hidden layer error and gradient

        #hiddenError= Ehidden= outputGradient . W hidden-output ...transpose
        hidden_error = output_delta.dot(self.weights_hidden_output.T)

        #hiddenGradient= Ehidden * sigmoidDerivative (hidden- layer-output)
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden_layer_output)

        # Update weights and biases
        
        #Update weights 
        self.weights_input_hidden += X.T.dot(hidden_delta) * learning_rate
        self.weights_hidden_output += self.hidden_layer_output.T.dot(output_delta) * learning_rate

        #Update biases
        self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * learning_rate

    # Training the ANN
    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, learning_rate)
            loss = mse_loss(y.reshape(-1, 1), output)
            if (epoch + 1) % 100 == 0:
                print(f"Epoch {epoch + 1}, Loss: {loss:.4f}")

In [148]:
# Define hyperparameters based on dataset
input_size = xtrain.shape[1]  # Number of features
hidden_size = 10  # Number of Neuron in Hidden Layer
output_size = 1   # Binary output (0 or 1)
learning_rate = 0.01
epochs = 1000

In [149]:
# Create and train the neural network
nn = ANN(input_size, hidden_size, output_size)
nn.train(xtrain, ytrain, epochs, learning_rate)

Epoch 100, Loss: 0.0286
Epoch 200, Loss: 0.0171
Epoch 300, Loss: 0.0143
Epoch 400, Loss: 0.0129
Epoch 500, Loss: 0.0120
Epoch 600, Loss: 0.0112
Epoch 700, Loss: 0.0106
Epoch 800, Loss: 0.0101
Epoch 900, Loss: 0.0097
Epoch 1000, Loss: 0.0094


In [161]:
# Testing the model on test data
print("\nTesting:")
predictions = nn.forward(xtest)
predictions = (predictions > 0.5).astype(int)  # Convert to binary output (0 or 1)


# Evaluate the accuracy
accuracy = np.mean(predictions.flatten() == ytest)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Testing:
Test Accuracy: 97.37%
